# Example of abox generation from a json file or python dictonary

Please have a look at section from the [official docs](https://data2rdf.readthedocs.io/en/latest/examples/abox/3_json.html) for more details.

## Import dependencies

In [72]:
from data2rdf import Data2RDF, Parser
import requests
import json

## Get raw data

In [73]:
file_name = "https://github.com/MI-FraunhoferIWM/data2rdf/raw/v2.0.0/tests/abox/json_pipeline_test/input/data/sample_data.json"

file = requests.get(file_name).text

print(file)

{
  "data": {
    "Breitenaenderung": {
      "unit": "mm",
      "value": 1.0
    },
    "Dehnung": [
      1.0,
      2.0,
      3.0
    ],
    "Standardkraft": {
      "array": [
        2.0,
        3.0,
        4.0
      ],
      "unit": "kN"
    }
  },
  "details": {
    "Bemerkungen": "foobar"
  }
}



## Define Mapping

In [74]:
mapping_file = "https://github.com/MI-FraunhoferIWM/data2rdf/raw/v2.0.0/tests/abox/json_pipeline_test/input/mapping/tensile_test_mapping.json"

mapping = requests.get(mapping_file).json()

print(requests.get(mapping_file).text)

[
  {
    "iri": "https://w3id.org/steel/ProcessOntology/Remark",
    "key": "Bemerkungen",
    "value_location": "details.Bemerkungen"
  },
  {
    "iri": "https://w3id.org/steel/ProcessOntology/WidthChange",
    "key": "Breitenaenderung",
    "unit_location": "data.Breitenaenderung.unit",
    "value_location": "data.Breitenaenderung.value"
  },
  {
    "iri": "https://w3id.org/steel/ProcessOntology/PercentageElongation",
    "key": "Dehnung",
    "unit": "%",
    "value_location": "data.Dehnung"
  },
  {
    "iri": "https://w3id.org/steel/ProcessOntology/Force",
    "key": "Standardkraft",
    "unit_location": "data.Standardkraft.unit",
    "value_location": "data.Standardkraft.array"
  }
]



## Run the pipeline

### Version A: as json file

In [75]:
pipeline = Data2RDF(
    raw_data=file,
    parser=Parser.json,
    mapping=mapping,
)

### Version B: as python dict

In [76]:
pipeline = Data2RDF(
    raw_data=json.loads(file),
    parser=Parser.json,
    mapping=mapping,
)

## Print graph

In [77]:
print(pipeline.graph.serialize())

@prefix dcat: <http://www.w3.org/ns/dcat#> .
@prefix dcterms: <http://purl.org/dc/terms/> .
@prefix fileid: <https://www.example.org/> .
@prefix foaf1: <http://xmlns.com/foaf/spec/> .
@prefix ns1: <prov:> .
@prefix qudt: <http://qudt.org/schema/qudt/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

fileid:dataset a dcat:Dataset ;
    dcterms:hasPart fileid:Dictionary ;
    dcat:distribution [ a dcat:Distribution ;
            dcat:accessURL "https://www.example.org/download"^^xsd:anyURI ;
            dcat:mediaType "https://www.iana.org/assignments/media-types/application/json"^^xsd:anyURI ] .

fileid:Dictionary a ns1:Dictionary ;
    ns1:hadDictionaryMember [ a ns1:KeyEntityPair ;
            ns1:pairEntity [ a ns1:Entity ;
                    qudt:quantity fileid:Force ;
                    foaf1:page [ a foaf1:Document ;
                            dcterms:format "https://www.iana.org/assignments/media-types/application/j

## Print parsed general metadata

In [78]:
pipeline.general_metadata

[PropertyGraph(
 	iri=https://w3id.org/steel/ProcessOntology/Remark,
 	suffix=Remark,
 	key=Bemerkungen,
 	value=foobar,
 	annotation=None,
 	value_relation=rdfs:label),
 QuantityGraph(
 	iri=https://w3id.org/steel/ProcessOntology/WidthChange,
 	suffix=WidthChange,
 	key=Breitenaenderung,
 	unit=http://qudt.org/vocab/unit/MilliM,
 	value=1.0,
 	unit_relation=qudt:hasUnit,
 	value_relation=qudt:value)]

## Print parsed dataframe metadata

In [79]:
pipeline.dataframe_metadata

[QuantityGraph(
 	iri=https://w3id.org/steel/ProcessOntology/PercentageElongation,
 	suffix=PercentageElongation,
 	key=Dehnung,
 	unit=http://qudt.org/vocab/unit/PERCENT,
 	value=None,
 	unit_relation=qudt:hasUnit,
 	value_relation=qudt:value),
 QuantityGraph(
 	iri=https://w3id.org/steel/ProcessOntology/Force,
 	suffix=Force,
 	key=Standardkraft,
 	unit=http://qudt.org/vocab/unit/KiloN,
 	value=None,
 	unit_relation=qudt:hasUnit,
 	value_relation=qudt:value)]

## Print parsed dataframe data

In [80]:
print(pipeline.dataframe)

   PercentageElongation  Force
0                   1.0    2.0
1                   2.0    3.0
2                   3.0    4.0
